In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Bidirectional
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
sns.set()
tf.compat.v1.random.set_random_seed(1234)

2023-10-09 17:30:35.579743: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 17:30:35.607331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 17:30:35.607359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 17:30:35.607376: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 17:30:35.611966: I tensorflow/core/platform/cpu_feature_g

In [2]:
df = pd.read_csv('../ml-models/dataset/SPY_2020-01-01_2022-01-01.csv')

# Adj Close Index= 5:6
minmax = MinMaxScaler().fit(df.iloc[:, 5:6].astype('float32'))
df_log = minmax.transform(df.iloc[:, 5:6].astype('float32'))
df_log = pd.DataFrame(df_log)

test_size = 30
simulation_size = 10

df_train = df_log.iloc[:-test_size]
df_test = df_log.iloc[-test_size:]
df.shape, df_train.shape, df_test.shape

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:507: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:507: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):


((505, 7), (475, 1), (30, 1))

In [5]:
num_layers = 1
size_layer = 128
timestamp = 1
epoch = 300
dropout_rate = 0.8
future_day = test_size
learning_rate = 0.01

In [4]:
class CustomModel(tf.keras.Model):
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias=0.1,
    ):
        super(CustomModel, self).__init__()
        self.gru_layers_forward = [GRU(size_layer, return_sequences=True, return_state=True) for _ in range(num_layers)]
        self.gru_layers_backward = [GRU(size_layer, return_sequences=True, return_state=True, go_backwards=True) for _ in range(num_layers)]
        self.dense = Dense(output_size)
        self.optimizer_instance = tf.keras.optimizers.Adam(learning_rate)

    def call(self, inputs, training=False):
        x, initial_state_fw, initial_state_bw = inputs
        states_fw = initial_state_fw
        states_bw = initial_state_bw
        for layer_fw, layer_bw in zip(self.gru_layers_forward, self.gru_layers_backward):
            x_fw, states_fw = layer_fw(x, initial_state=states_fw)
            x_bw, states_bw = layer_bw(x, initial_state=states_bw)
        x = tf.concat([x_fw, x_bw], axis=-1)
        logits = self.dense(x)
        return logits

    def train_step(self, data):
        x, y, initial_state_fw, initial_state_bw = data
        with tf.GradientTape() as tape:
            logits = self((x, initial_state_fw, initial_state_bw), training=True)
            loss = tf.reduce_mean(tf.square(y - logits))
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer_instance.apply_gradients(zip(gradients, self.trainable_variables))
        return {'loss': loss}

def calculate_accuracy(real, predict):
    real = np.array(real) + 1
    predict = np.array(predict) + 1
    percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
    return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer


In [6]:
def forecast():
    modelnn = CustomModel(
        learning_rate, num_layers, df_log.shape[1], size_layer, df_log.shape[1], dropout_rate
    )

    modelnn.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss='mean_squared_error'  # Assuming you're using Mean Squared Error loss
    )

    date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()

    pbar = tqdm(range(epoch), desc='train loop')
    for i in pbar:
        init_value_forward = np.zeros((1, num_layers * size_layer))
        init_value_backward = np.zeros((1, num_layers * size_layer))
        total_loss, total_acc = [], []
        for k in range(0, df_train.shape[0] - 1, timestamp):
            index = min(k + timestamp, df_train.shape[0] - 1)
            batch_x = np.expand_dims(
                df_train.iloc[k : index, :].values, axis = 0
            )
            batch_y = df_train.iloc[k + 1 : index + 1, :].values
            history = modelnn.train_on_batch(
                [batch_x, init_value_forward, init_value_backward],
                batch_y
            )
            logits = modelnn.predict_on_batch([batch_x, init_value_forward, init_value_backward])
            loss = history['loss']
            total_loss.append(loss)
            total_acc.append(calculate_accuracy(batch_y[:, 0], logits[:, 0]))
        pbar.set_postfix(cost=np.mean(total_loss), acc=np.mean(total_acc))
    

    future_day = test_size
    output_predict = np.zeros((df_train.shape[0] + future_day, df_train.shape[1]))
    output_predict[0] = df_train.iloc[0]
    upper_b = (df_train.shape[0] // timestamp) * timestamp
    init_value_forward = np.zeros((1, num_layers * size_layer))
    init_value_backward = np.zeros((1, num_layers * size_layer))

    for k in range(0, (df_train.shape[0] // timestamp) * timestamp, timestamp):
        out_logits = modelnn.predict_on_batch(
            [np.expand_dims(df_train.iloc[k: k + timestamp], axis=0),
             init_value_forward, init_value_backward]
        )
        # Assume that modelnn.call or modelnn.predict_on_batch returns logits and state as a tuple
        init_value_forward, init_value_backward = out_logits[1]
        output_predict[k + 1: k + timestamp + 1] = out_logits[0]

    if upper_b != df_train.shape[0]:
        out_logits = modelnn.predict_on_batch(
            [np.expand_dims(df_train.iloc[upper_b:], axis=0),
             init_value_forward, init_value_backward]
        )
        init_value_forward, init_value_backward = out_logits[1]
        output_predict[upper_b + 1: df_train.shape[0] + 1] = out_logits[0]
        future_day -= 1
        date_ori.append(date_ori[-1] + timedelta(days=1))

    init_value_forward = out_logits[1][0]
    init_value_backward = out_logits[1][1]

    for i in range(future_day):
        o = output_predict[-future_day - timestamp + i:-future_day + i]
        out_logits = modelnn.predict_on_batch(
            [np.expand_dims(o, axis=0),
             init_value_forward, init_value_backward]
        )
        init_value_forward, init_value_backward = out_logits[1]
        output_predict[-future_day + i] = out_logits[0][-1]
        date_ori.append(date_ori[-1] + timedelta(days=1))

    output_predict = minmax.inverse_transform(output_predict)
    deep_future = anchor(output_predict[:, 0], 0.3)

    return deep_future[-test_size:]

In [7]:


results = []
for i in range(1):
    print('simulation %d'%(i + 1))
    results.append(forecast())



simulation 1


2023-10-09 17:30:53.228473: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-09 17:30:53.236137: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-09 17:30:53.236319: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

ValueError: in user code:

    File "/home/andrea/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/andrea/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/andrea/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/tmp/ipykernel_111320/1922925663.py", line 29, in train_step
        x, y, initial_state_fw, initial_state_bw = data

    ValueError: not enough values to unpack (expected 4, got 2)
